In [119]:
import sys; sys.path.append('../../')
import numpy as np
from DataPreparation.DataPreparation import read_data

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from mlpath import mlquest as mlq

mlq.start_quest("LogisticRegression", table_dest="../../", log_defs=False)

/opt/anaconda3/envs/arm/lib/python3.9/site-packages/mlpath/mlquest/mlquest.py:66: UserWarning: Attempting to start a run while another one is active may cause data overwrite
  if mlquest.active == True: warnings.warn("Attempting to start a run while another one is active may cause data overwrite")


### Read the data

In [120]:
x_data_d, y_data_d = mlq.l(read_data)(kind='Numerical')

### Initiate model

In [123]:
clf = mlq.l(LogisticRegression)(multi_class='multinomial')

### Train and Evaluate

In [31]:
y_pred = mlq.l(cross_val_predict)(clf, x_data_d, y_data_d, cv=4)
accuracy = np.mean(y_pred == y_data_d)
mlq.log_metrics(accuracy=accuracy.item())
print(classification_report(y_data_d, y_pred))
mlq.end_quest()

              precision    recall  f1-score   support

           0       0.27      0.22      0.25       152
           1       0.13      0.03      0.05       160
           2       0.36      0.01      0.02       324
           3       0.52      0.96      0.67       544

    accuracy                           0.48      1180
   macro avg       0.32      0.31      0.25      1180
weighted avg       0.39      0.48      0.36      1180



### Tracking

In [15]:
mlq.show_logs("LogisticRegression", table_dest="../../", last_k=6)